In [ ]:
num_sample = 300
num_burn = 100
sample_size = 800

In [ ]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
min_degree = 1
max_degree = 1
adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())
adj = np.zeros((sample_size, sample_size))

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1,2,0.1,2,0.1,-2,0.1,2,0.1,0])

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

1 9


100%|██████████| 400/400 [00:00<00:00, 554.02it/s]


(0.6274429054981997, 0.35146249999999996)

In [ ]:
import networkx as nx
def compute_avg_effects_std_from_raw(psi_vec, adj_matrix, h=2):
    N = len(psi_vec)
    avg_effects = np.mean(psi_vec)
    
    # Centered residuals
    g = psi_vec - avg_effects
    
    # Build graph and compute pairwise distances
    G = nx.from_numpy_array(adj_matrix)
    dist = dict(nx.all_pairs_shortest_path_length(G, cutoff=h))
    
    # Network HAC estimator
    hac_var = 0.0
    for i in range(N):
        for j, dij in dist[i].items():
            weight = max(1 - dij / h, 0)  # Bartlett kernel
            hac_var += weight * g[i] * g[j]
    
    hac_var /= N
    se_hac = np.sqrt(hac_var)/np.sqrt(N)

    return avg_effects, se_hac

In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i,
         'treatment_allocation': 0.7, 
         'psi_0_gamma_only': False,
         'mispec': None} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=10)
res_list_array_dr = np.array(res_list_dr)

Multiprocessing <function run_dr_raw at 0x138aecc10> in 100 tasks, with 10 processes...


Process SpawnPoolWorker-8:
Process SpawnPoolWorker-7:
Process SpawnPoolWorker-3:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/run_pll.py", line 49, in run_dr_raw
    ret_i = doubly_robust(A_chain[i], L_chain[i], Y_chain[i], adj, treatment_allocation=treatment_allocation, seed=1, return_raw=True,
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/drnet.py", line 232, in doubly_robust
    numerator, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, neighbours_2hop, gamma, adj_matrix, Atype='ind_treat_0')
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/drnet.py", lin

KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm

var = []
var_sim = []
est = []
idx = 0
for i in tqdm(range(Y_chain.shape[0])):
    avg_effects, se_hac = compute_avg_effects_std_from_raw(res_list_dr[i,:,0],adj,h=3)
    var.append(se_hac**2*sample_size)
    var_sim.append(np.var(res_list_dr[i,:,0]))
    est.append(avg_effects)

var_true = np.var(est)
print(np.mean(var), np.mean(var_sim), var_true*sample_size)

  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: list indices must be integers or slices, not tuple

In [ ]:
# compute coverage rate for mean(Y)

mean_est = np.mean(res_list_dr, axis=1)
#std_est = np.sqrt(np.array(var))
std_est = np.std(res_list_dr, axis=1)
se_est = std_est / np.sqrt(res_list_dr.shape[1])
lower_bound = mean_est - 1.96*se_est
upper_bound = mean_est + 1.96*se_est
truth = np.mean(res_list_dr)
coverage_rate = np.mean((lower_bound < truth) & (upper_bound > truth))
print("Coverage rate for mean(Y):", coverage_rate)

Coverage rate for mean(Y): 0.8722


In [ ]:
np.sqrt(np.array(var)).mean(), np.std(Y_chain, axis=1).mean(), np.mean(var), np.var(Y_chain, axis=1).mean()

(0.4794930129108512,
 0.36706748125934646,
 0.2309678569227906,
 0.13497900359374998)

In [ ]:
k = 51
print(np.where(adj[k])[0])
for j in np.where(adj[k])[0]:
    print(np.corrcoef(L_chain[:,k,0], L_chain[:,j,0])[0,1])

ic = []
for j in np.where(adj[0]==0)[0]:
    ic.append(np.corrcoef(L_chain[:,k,0], L_chain[:,j,0])[0,1])
np.mean(ic)

[36 44 48 50 53 54 57 62]
-0.0024047659344970016
-0.006200550184607464
0.0002895150593442676
-0.0058514365867054765
0.003913545804805268
0.009396890404550115
-0.005531425003369036
-0.013987499018662464


0.0017564936591066714

In [ ]:
L_chain.shape, Y_chain.shape

((10000, 800, 3), (10000, 800))